In [3]:
# dataloader
import os
from lib.env import DATA_PATH
import json
import pandas as pd
from lib.utils import *
import matplotlib.pyplot as plt
import plotly.express as px
from torch.utils.data import DataLoader,TensorDataset,ConcatDataset
import torch
from sklearn.metrics import ConfusionMatrixDisplay,classification_report

FS = 50
WINDOWSIZE = 100 * FS

In [4]:
from sage.models import ResNetv2,ResBlockv2
from tqdm import tqdm
from sage.utils import evaluate_sigmoid

hyperparameters = {
    'block':ResBlockv2,
    'widthi':[4],
    'depthi':[2],
    'n_output_neurons':1,
    'norm':'layer'
}

traindataset = ConcatDataset([
    get_dataset_for_project("ejaz_phase1",windowsize=WINDOWSIZE),
    get_dataset_for_project("ejaz_phase2",windowsize=WINDOWSIZE),
    # get_dataset_for_project("iftakhar_phase1",windowsize=WINDOWSIZE),
    # get_dataset_for_project("iftakhar_phase2",windowsize=WINDOWSIZE),
    ])

# traindataset = get_dataset_for_project("alsaad_phase1",windowsize=WINDOWSIZE)
devdataset = get_dataset_for_project("alsaad_phase1",windowsize=WINDOWSIZE)
testdataset = get_dataset_for_project("alsaad_phase2",windowsize=WINDOWSIZE)

In [ ]:
train_losses = []
val_losses = []
test_losses = []
train_f1_scores = []
val_f1_scores = []
test_f1_scores = []
model = ResNetv2(**hyperparameters)
optimizer = torch.optim.AdamW(model.parameters(),lr=3e-4)
criterion = torch.nn.BCEWithLogitsLoss()
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=50)

In [8]:
# trainloader = DataLoader(traindataset,batch_size=128,shuffle=True)
trainloader = DataLoader(ConcatDataset([traindataset,devdataset]),batch_size=128,shuffle=True)
devloader = DataLoader(devdataset,batch_size=128,shuffle=True)
testloader = DataLoader(testdataset,batch_size=128,shuffle=True)

In [ ]:
import matplotlib.pyplot as plt
import torch
from sklearn.metrics import f1_score

# Set the interval for validation evaluation (e.g., every 10 batches)
val_interval = 10

# Move the model to GPU (if available)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    train_preds, train_labels = [], []
    
    for i, (inputs, labels) in enumerate(trainloader):

        # Evaluate the validation and test sets every `val_interval` batches
        if (i + 1) % val_interval == 0:
            model.eval()

            # Validation evaluation
            val_loss = 0.0
            val_preds, val_labels_list = [], []
            with torch.no_grad():
                for val_inputs, val_labels in devloader:
                    # Move inputs and labels to the GPU
                    val_inputs, val_labels = val_inputs.to(device), val_labels.to(device)
                    
                    val_outputs = model(val_inputs)
                    val_loss += criterion(val_outputs, val_labels).item()

                    # Store validation predictions and labels
                    val_preds.extend(torch.sigmoid(val_outputs).round().cpu().numpy())
                    val_labels_list.extend(val_labels.cpu().numpy())

            val_loss /= len(devloader)  # Average validation loss
            val_losses.append(val_loss)

            # Test evaluation
            test_loss = 0.0
            test_preds, test_labels_list = [], []
            with torch.no_grad():
                for test_inputs, test_labels in testloader:
                    # Move inputs and labels to the GPU
                    test_inputs, test_labels = test_inputs.to(device), test_labels.to(device)
                    
                    test_outputs = model(test_inputs)
                    test_loss += criterion(test_outputs, test_labels).item()

                    # Store test predictions and labels
                    test_preds.extend(torch.sigmoid(test_outputs).round().cpu().numpy())
                    test_labels_list.extend(test_labels.cpu().numpy())

            test_loss /= len(testloader)  # Average test loss
            test_losses.append(test_loss)

            # Compute F1 scores


            print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(trainloader)}], '
                  f'Train Loss: {running_loss / val_interval:.4f}, Val Loss: {val_loss:.4f}, '
                  f'Test Loss: {test_loss:.4f}, Train F1: {train_f1:.4f}, Val F1: {val_f1:.4f}, Test F1: {test_f1:.4f}')
            
            # Reset running loss and F1 calculation data
            train_losses.append(running_loss / val_interval)
            running_loss = 0.0
            train_preds, train_labels = [], []  # Reset for the next round
            
            model.train()  # Switch back to training mode

            # Plotting losses and F1 scores in subplots
            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

            # Loss plot
            ax1.plot(train_losses, label='Train Loss')
            ax1.plot(val_losses, label='Validation Loss')
            ax1.plot(test_losses, label='Test Loss')
            ax1.set_xlabel('Steps (scaled by interval)')
            ax1.set_ylabel('Loss')
            ax1.set_title('Loss Curves')
            ax1.legend()

            # F1 Score plot
            ax2.plot(train_f1_scores, label='Train F1 Score')
            ax2.plot(val_f1_scores, label='Validation F1 Score')
            ax2.plot(test_f1_scores, label='Test F1 Score')
            ax2.set_xlabel('Steps (scaled by interval)')
            ax2.set_ylabel('F1 Score')
            ax2.set_title('F1 Score Curves')
            ax2.legend()

            plt.tight_layout()
            plt.savefig('training_validation_test_curves.jpg')
            plt.close()

In [ ]:
torch.save(model.state_dict(),'custom.pt')

In [ ]:
loss,y_true,y_pred = evaluate_sigmoid(trainloader,model,criterion)
ConfusionMatrixDisplay.from_predictions(y_true,y_pred)
print(classification_report(y_true,y_pred))
loss,y_true,y_pred = evaluate_sigmoid(devloader,model,criterion)
ConfusionMatrixDisplay.from_predictions(y_true,y_pred)
print(classification_report(y_true,y_pred))
loss,y_true,y_pred = evaluate_sigmoid(testloader,model,criterion)
ConfusionMatrixDisplay.from_predictions(y_true,y_pred)
print(classification_report(y_true,y_pred))